In [5]:
import torch
from torch.nn import functional as F
import numpy as np

## [FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/abs/2205.14135)

### Forward Pass

In [323]:
'''
    Q: (batch, seq_len, dim)
    K: (batch, seq_len, dim)
    V: (batch, seq_len, dim)
    tau: softmax scaling
    is_causal: whether to use causal mask
    dropout_p: dropout probability
    
    return: (batch, seq_len, dim)
'''

def flash_attention(Q, K, V, M = 100_000, tau = 1, is_causal = False, dropout_p = 0, seed = 1234):
    b, N, d = Q.shape
    Bc = M // (4 * d)
    Br = min(Bc,d)
    O = torch.zeros_like(Q)
    l = torch.zeros(b, N)
    m = torch.ones(b, N) * -float('inf')
    Tr = N // Br + (1 if N % Br != 0 else 0)
    Tc = N // Bc + (1 if N % Bc != 0 else 0)

    for b_i in range(b):
        for j in range(Tc):
            N_c = Bc * j
            K_j = K[b_i,N_c:N_c + Bc]
            V_j = V[b_i,N_c:N_c + Bc]
            
            for i in range(Tr):
                N_r = Br * i
                Q_i = Q[b_i, N_r:N_r + Br]
                O_i = O[b_i, N_r:N_r + Br]
                l_i = l[b_i, N_r:N_r + Br]
                m_i = m[b_i, N_r:N_r + Br]

                S_ij = tau * (Q_i @ K_j.T)
                
                if is_causal:
                    # Apply causal mask
                    r_max = N_c + Bc - N_r - 1
                    c_start = Bc - r_max
                    for r in range(min(r_max, S_ij.shape[0])):
                        S_ij[r, max(c_start + r, 0):] = torch.tensor(-float('inf'))

                m_ij = torch.max(S_ij, dim=1).values
                S_ij = S_ij - m_ij.unsqueeze(1)
                # Need to make sure that the exp is not nan, value of nan could be caused by applying causal mask
                S_ij[S_ij.isnan()] = 0
                P_ij = torch.exp(S_ij)
                # P_ij = torch.exp(torch.where(S_ij - m_ij.unsqueeze(1) != float('nan'), ))
                l_ij = torch.sum(P_ij, dim=1)
                
                m_i_new = torch.max(m_i, m_ij)
                l_i_new = torch.exp(m_i - m_i_new) * l_i + torch.exp(m_ij - m_i_new) * l_ij
                if dropout_p > 0:
                    torch.manual_seed(seed)
                    P_ij = F.dropout(P_ij, p = dropout_p, training = True)
                
                O_i = torch.diag(1 / l_i_new) @ (torch.diag(l_i * torch.exp(m_i - m_i_new)) @ O_i
                                                 + torch.diag(torch.exp(m_ij - m_i_new)) @ (P_ij @ V_j))

                O[b_i, N_r:N_r + Br] = O_i
                l[b_i, N_r:N_r + Br] = l_i_new
                m[b_i, N_r:N_r + Br] = m_i_new
               
    return O, l, m

def attention(Q, K, V, tau = 1, dropout_p = 0, is_causal = False):
    S = (Q @ K.transpose(1,2)) * tau
    if is_causal:
        S = torch.where(torch.tril(torch.ones_like(S)) != 0, S, torch.tensor(-float('inf')))  # apply causal mask
    P = torch.softmax(S, dim=-1)
    if dropout_p > 0:
        P = F.dropout(P, p = dropout_p, training = True)

    O = P @ V

    return O, P

#### Test

In [326]:
%%time

b, n, d = 16, 1024, 512
# b, n, d = 8, 128, 32

Q = torch.randn(b, n, d)
K = torch.randn(b, n, d)
V = torch.randn(b, n, d)

O, P = attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True)
O_flash, l, m = flash_attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True)


print(O[0][0][:10])
print(O_flash[0][0][:10])
print(f'Total difference: {torch.sum(torch.abs(O_flash - O))}')
print(O_flash.allclose(O, atol=1e-4))

O_flash = flash_attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True, dropout_p = 0.1)
print(O_flash[0][0][:10])

tensor([ 0.0955,  0.6573, -1.4128, -0.6750,  0.7921, -0.4240,  0.4224, -1.4545,
        -0.2952,  0.8288])
tensor([ 0.0955,  0.6573, -1.4128, -0.6750,  0.7921, -0.4240,  0.4224, -1.4545,
        -0.2952,  0.8288])
Total difference: 0.30684229731559753
True
tensor([[ 0.1061,  0.7303, -1.5698,  ...,  0.4672,  1.0930,  1.3335],
        [-0.1154,  0.1522, -0.3652,  ...,  0.0652,  0.7005,  0.2060],
        [ 0.0710,  0.2778, -0.9313,  ...,  0.3003,  0.4251,  0.4335],
        ...,
        [ 1.1259,  0.9318, -1.2136,  ...,  0.5742, -0.5662,  0.3515],
        [ 0.7906, -0.4637, -0.4523,  ...,  0.0695, -0.3271, -0.2006],
        [ 0.8441,  0.4677, -0.6430,  ...,  0.1067, -0.3857,  0.2001]])
CPU times: user 1min 10s, sys: 419 ms, total: 1min 11s
Wall time: 7.24 s


### Backward Pass

In [450]:
def flash_attention_grad(Q, K, V, O, dO, l, m, M = 100_000, tau = 1, is_causal = False, dropout_p = 0, seed = 1234):
    b, N, d = Q.shape
    Bc = M // (4 * d)
    Br = min(Bc,d)
    Tr = N // Br + (1 if N % Br != 0 else 0)
    Tc = N // Bc + (1 if N % Bc != 0 else 0)
    
    dQ = torch.zeros_like(Q)
    dK = torch.zeros_like(K)
    dV = torch.zeros_like(V)

    for b_i in range(b):
        for j in range(Tc):
            N_c = Bc * j
            K_j = K[b_i,N_c:N_c + Bc]
            V_j = V[b_i,N_c:N_c + Bc]
            dK_j = dK[b_i,N_c:N_c + Bc]
            dV_j = dV[b_i,N_c:N_c + Bc]
            
            for i in range(Tr):
                N_r = Br * i
                Q_i = Q[b_i, N_r:N_r + Br]
                O_i = O[b_i, N_r:N_r + Br]
                dQ_i = dQ[b_i, N_r:N_r + Br]
                dO_i = dO[b_i, N_r:N_r + Br]
                l_i = l[b_i, N_r:N_r + Br]
                m_i = m[b_i, N_r:N_r + Br]

                S_ij = tau * (Q_i @ K_j.T)

                if is_causal:
                    # Apply causal mask
                    r_max = N_c + Bc - N_r - 1
                    c_start = Bc - r_max
                    for r in range(min(r_max, S_ij.shape[0])):
                        S_ij[r, max(c_start + r, 0):] = torch.tensor(-float('inf'))
                
                S_ij = S_ij - m_i.unsqueeze(1)
                # Need to make sure that the exp is not nan, value of nan could be caused by applying causal mask
                S_ij[S_ij.isnan()] = 0
                P_ij = torch.diag(1 / l_i) @ torch.exp(S_ij)
                
                Z_ij = torch.ones_like(P_ij)
                if dropout_p > 0:
                    torch.manual_seed(seed)
                    Z_ij = F.dropout(Z_ij, p = dropout_p, training = True)
                    P_ij = Z_ij * P_ij

                dV_j += P_ij.T @ dO_i

                dP_ij = (dO_i @ V_j.T) * Z_ij
                D_i = torch.sum(dO_i * O_i, dim=1)
                dS_ij = P_ij * (dP_ij - D_i.unsqueeze(1))
                
                dQ[b_i, N_r:N_r + Br] = dQ_i + tau * (dS_ij @ K_j )
                
                dK_j += tau * (dS_ij.T @ Q_i)
            
        dK[b_i,N_c:N_c + Bc] = dK_j
        dV[b_i,N_c:N_c + Bc] = dV_j


    return dQ, dK, dV


def attention_grad(Q, K, V, P, dO):
    dV = P.transpose(1,2) @ dO
    dP = dO @ V.transpose(1, 2)
    dS = P * (dP - (P * dP).sum(dim=-1).unsqueeze(-1))
    # dS = torch.zeros_like(dP)
    # for i in range(dP.shape[1]):
    #     for j in range(dP.shape[2]):
    #         dS[:, i, j] = P[:, i, j] * (dP[:, i, j] - (P[:,i] * dP[:,i]).sum(dim=1))       
    dQ = dS @ K
    dK = dS.transpose(1,2) @ Q

    return dQ, dK, dV

#### Test

In [451]:
torch.manual_seed(0)
# b, n, d = 16, 1024, 512
b, n, d = 8, 128, 32

Q = torch.randn(b, n, d)
K = torch.randn(b, n, d)
V = torch.randn(b, n, d)

O, P = attention(Q, K, V, tau=1/np.sqrt(d), is_causal = False)
O_flash, l_flash, m_flash = flash_attention(Q, K, V, tau=1/np.sqrt(d), is_causal = False)
print(O_flash.allclose(O, atol=1e-4))

dO = O * 0.1
# dO = torch.randn(b, n, d)

dQ, dK, dV = attention_grad(Q, K, V, P, dO)
dQ_flash, dK_flash, dV_flash = flash_attention_grad(Q, K, V, O, dO, l_flash, m_flash, tau=1/np.sqrt(d), is_causal = False)
print(dQ[0][0][:10], dQ_flash[0][0][:10])
print(dK[0][0][:10], dK_flash[0][0][:10])
print(dV.allclose(dV_flash, atol=1e-4))

True
tensor([-0.0055,  0.0057, -0.0039, -0.0091,  0.0157, -0.0019, -0.0066, -0.0072,
         0.0027, -0.0008]) tensor([-0.0010,  0.0010, -0.0007, -0.0016,  0.0028, -0.0003, -0.0012, -0.0013,
         0.0005, -0.0001])
tensor([-0.0296, -0.0018, -0.0439, -0.0665, -0.0246,  0.0578, -0.0130, -0.1125,
         0.0191, -0.0237]) tensor([-0.0052, -0.0003, -0.0078, -0.0118, -0.0043,  0.0102, -0.0023, -0.0199,
         0.0034, -0.0042])
True
